In [119]:
import pandapower as pp
from math import acos,tan

In [120]:
#use 11kV and use cleveland, 95 smaller diameter
#assume inductance is zero/negligible, because close together
#need to find capacity
subnet = pp.create_empty_network()

V_nom_kV = 0.240*(3)**(1/2)
V_intermediate_kV=11
V_high_kV=132
buses= []

r_m_high=0.320
#or 0.411
x_m_high=0.120
c_high=0.32
max_i_high=0.228

r_m_low=1.15 #1.600 #2.5
x_m_low=0.08 #0.140
c_low=0.20#1.650
max_i_low=0.75 #0.096 #0.075


#create the 9 buses, which are at the 0.415 voltage
for i in range(8):
    buses.append(pp.create_bus(subnet,vn_kv=V_nom_kV, name=f'Bus{i}'))

#high voltage bus for trafo
buses.append(pp.create_bus(subnet, vn_kv=V_intermediate_kV, name='Bus 8 (Trafo intermediate)'))

#create buses 9 to 11 at 11kv
for i in range(9,12):
    buses.append(pp.create_bus(subnet,vn_kv=V_intermediate_kV, name=f'Bus dummy trafto intermediate {i}'))

#create buses 12 to 14 at 11kv
for i in range(12,15):
    buses.append(pp.create_bus(subnet,vn_kv=V_nom_kV, name=f'Bus dummy trafto nom {i}'))
    
#create buses 15 and 16
buses.append(pp.create_bus(subnet,vn_kv=V_intermediate_kV, name='Bus Trafo from grid/11kV'))
buses.append(pp.create_bus(subnet,vn_kv=V_high_kV, name='Bus Trafo from grid/132kV'))

# bus 16 now becomes an external grid
pp.create_ext_grid(subnet,bus=buses[16],vm_pu=1,name='Swing bus(Grid Connection)')
 


    
    
    
#create transformers
#43 houses per trafo

admd=0.003
#safety_factor=1.5 #give it a higher rate load
#sn_mva43 =43*admd*1.5 #rated apparent power
#sn_mva1 =admd*1.5 #rated apparent power
#sn_mva172=172*admd*1.5


#the one branch
pp.create_transformer_from_parameters(net=subnet, hv_bus=buses[8], lv_bus=buses[0], sn_mva=1.5, vn_hv_kv=V_intermediate_kV,vn_lv_kv=V_nom_kV,vkr_percent=1,vk_percent=10,pfe_kw=1.5/100*1000,i0_percent=5) 
#the other 3
for i in range(9,12):
    pp.create_transformer_from_parameters(net=subnet, hv_bus=buses[i], lv_bus=buses[i+3], sn_mva=1.5, vn_hv_kv=V_intermediate_kV,vn_lv_kv=V_nom_kV,vkr_percent=1,vk_percent=10,pfe_kw=1.5/100*1000,i0_percent=5) 

#the big one
pp.create_transformer_from_parameters(net=subnet, hv_bus=buses[16], lv_bus=buses[15], sn_mva=1.5, vn_hv_kv=V_high_kV,vn_lv_kv=V_intermediate_kV,vkr_percent=1,vk_percent=10,pfe_kw=1.5/100*1000,i0_percent=5) 

    

    
    
    
    
    


#create the loads

loads=[]
power_factor=0.9

p_mw_per_house=0.003
q_mvar_per_house=p_mw_per_house*tan(acos(power_factor))
p_mw_PV_per_house=-0.0012
q_mvar_PV_per_house=p_mw_PV_per_house*tan(acos(power_factor))
number_of_houses_normal=6
number_of_houses_extra=7


for i in range(1,7):
    loads.append(pp.create_load(subnet,bus=buses[i], p_mw=p_mw_per_house*number_of_houses_normal, q_mvar=q_mvar_per_house*number_of_houses_normal,  name=f'Combined load at bus{i}'))
    #loads.append(pp.create_load(subnet,bus=buses[i], p_mw=p_mw_PV_per_house*number_of_houses_normal, q_mvar=q_mvar_PV_per_house*number_of_houses_normal,  name=f'Combined PV at bus{i}'))
#For the 7 house bus
loads.append(pp.create_load(subnet,bus=buses[7], p_mw=p_mw_per_house*number_of_houses_extra, q_mvar=q_mvar_per_house*number_of_houses_extra,  name=f'Combined load at bus{i}'))
#loads.append(pp.create_load(subnet,bus=buses[7], p_mw=p_mw_PV_per_house*number_of_houses_extra, q_mvar=q_mvar_PV_per_house*number_of_houses_extra,  name=f'Combined PV at bus{i}'))


#dummy loads
for i in range(12,15):
    loads.append(pp.create_load(subnet,bus=buses[i], p_mw=p_mw_per_house*43, q_mvar=q_mvar_per_house*43,  name=f'Combined dummy load at bus{i}'))
    #loads.append(pp.create_load(subnet,bus=buses[i], p_mw=p_mw_PV_per_house*43, q_mvar=q_mvar_PV_per_house*43,  name=f'Combined dummy PV at bus{i}'))


#create storage
#pp.create_storage(net=subnet, bus=buses[0], p_mw=p_mw_PV_per_house*43,max_e_mwh=1.500)
#loads.append(pp.create_load(subnet,bus=buses[0], p_mw=p_mw_PV_per_house*43, q_mvar=q_mvar_PV_per_house*43,  name=f'Combined dummy PV at bus{i}'))

#create lines for branch
line_lengths=[0.05+0.075,0.150]

#left branch
for i in range(0,2):
    #remember to update later the actual values for x and q
    pp.create_line_from_parameters(subnet,from_bus=buses[i],to_bus=buses[i+1],length_km=line_lengths[i], r_ohm_per_km = r_m_low, x_ohm_per_km =x_m_low, c_nf_per_km = c_low, max_i_ka=max_i_low)

#create from 0 to 3 and 3 to 4
    
pp.create_line_from_parameters(subnet,from_bus=buses[0],to_bus=buses[3],length_km=line_lengths[0], r_ohm_per_km = r_m_low, x_ohm_per_km = x_m_low, c_nf_per_km = c_low, max_i_ka=max_i_low)
pp.create_line_from_parameters(subnet,from_bus=buses[3],to_bus=buses[4],length_km=line_lengths[1], r_ohm_per_km = r_m_low, x_ohm_per_km = x_m_low, c_nf_per_km = c_low, max_i_ka=max_i_low)

    
#right branch 

#connection from 0 to 5
pp.create_line_from_parameters(subnet,from_bus=buses[0],to_bus=buses[5],length_km=line_lengths[0], r_ohm_per_km = r_m_low, x_ohm_per_km = x_m_low, c_nf_per_km = c_low, max_i_ka=max_i_low)
#connection from 5 to 6
pp.create_line_from_parameters(subnet,from_bus=buses[5],to_bus=buses[6],length_km=line_lengths[1], r_ohm_per_km = r_m_low, x_ohm_per_km = x_m_low, c_nf_per_km = c_low, max_i_ka=max_i_low)
#connection from 0 to 7
pp.create_line_from_parameters(subnet,from_bus=buses[0],to_bus=buses[7],length_km=line_lengths[0], r_ohm_per_km = r_m_low, x_ohm_per_km = x_m_low, c_nf_per_km = c_low, max_i_ka=max_i_low)

#create rest of lines-assuming hypotenuse
long_lines=[0.495,0.300,0.300,0.495]
for i in range(8,12):
    pp.create_line_from_parameters(subnet,from_bus=buses[15],to_bus=buses[i],length_km=long_lines[i-8], r_ohm_per_km =r_m_high, x_ohm_per_km =x_m_high, c_nf_per_km = c_high, max_i_ka=max_i_high)



pp.runpp(subnet)


In [121]:
subnet.bus

,name,vn_kv,type,zone,in_service
0,Bus0,0.415692,b,None,True
1,Bus1,0.415692,b,None,True
2,Bus2,0.415692,b,None,True
3,Bus3,0.415692,b,None,True
4,Bus4,0.415692,b,None,True
5,Bus5,0.415692,b,None,True
6,Bus6,0.415692,b,None,True
7,Bus7,0.415692,b,None,True
8,Bus 8 (Trafo intermediate),11.000000,b,None,True
9,Bus dummy trafto intermediate 9,11.000000,b,None,True


In [122]:
subnet.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,None,None,0,1,0.125,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
1,None,None,1,2,0.150,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
2,None,None,0,3,0.125,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
3,None,None,3,4,0.150,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
4,None,None,0,5,0.125,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
5,None,None,5,6,0.150,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
6,None,None,0,7,0.125,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
7,None,None,15,8,0.495,0.32,0.12,0.32,0.0,0.228,1.0,1,None,True
8,None,None,15,9,0.300,0.32,0.12,0.32,0.0,0.228,1.0,1,None,True
9,None,None,15,10,0.300,0.32,0.12,0.32,0.0,0.228,1.0,1,None,True


In [123]:
subnet.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,Combined load at bus1,1,0.018,0.008718,0.0,0.0,NaN,1.0,True,wye
1,Combined load at bus2,2,0.018,0.008718,0.0,0.0,NaN,1.0,True,wye
2,Combined load at bus3,3,0.018,0.008718,0.0,0.0,NaN,1.0,True,wye
3,Combined load at bus4,4,0.018,0.008718,0.0,0.0,NaN,1.0,True,wye
4,Combined load at bus5,5,0.018,0.008718,0.0,0.0,NaN,1.0,True,wye
5,Combined load at bus6,6,0.018,0.008718,0.0,0.0,NaN,1.0,True,wye
6,Combined load at bus6,7,0.021,0.010171,0.0,0.0,NaN,1.0,True,wye
7,Combined dummy load at bus12,12,0.129,0.062478,0.0,0.0,NaN,1.0,True,wye
8,Combined dummy load at bus13,13,0.129,0.062478,0.0,0.0,NaN,1.0,True,wye
9,Combined dummy load at bus14,14,0.129,0.062478,0.0,0.0,NaN,1.0,True,wye


In [124]:
subnet.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,None,None,0,1,0.125,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
1,None,None,1,2,0.150,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
2,None,None,0,3,0.125,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
3,None,None,3,4,0.150,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
4,None,None,0,5,0.125,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
5,None,None,5,6,0.150,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
6,None,None,0,7,0.125,1.15,0.08,0.20,0.0,0.750,1.0,1,None,True
7,None,None,15,8,0.495,0.32,0.12,0.32,0.0,0.228,1.0,1,None,True
8,None,None,15,9,0.300,0.32,0.12,0.32,0.0,0.228,1.0,1,None,True
9,None,None,15,10,0.300,0.32,0.12,0.32,0.0,0.228,1.0,1,None,True


In [125]:
subnet.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.038513,0.017610,-0.036592,-0.017477,0.001921,0.000134,0.066734,0.066734,0.066734,0.881356,-8.339450,0.843967,-7.382638,8.897870
1,0.018592,0.008759,-0.018000,-0.008718,0.000592,0.000041,0.033822,0.033822,0.033822,0.843967,-7.382638,0.821302,-6.766590,4.509550
2,0.038513,0.017610,-0.036592,-0.017477,0.001921,0.000134,0.066734,0.066734,0.066734,0.881356,-8.339450,0.843967,-7.382638,8.897870
3,0.018592,0.008759,-0.018000,-0.008718,0.000592,0.000041,0.033822,0.033822,0.033822,0.843967,-7.382638,0.821302,-6.766590,4.509550
4,0.038513,0.017610,-0.036592,-0.017477,0.001921,0.000134,0.066734,0.066734,0.066734,0.881356,-8.339450,0.843967,-7.382638,8.897870
5,0.018592,0.008759,-0.018000,-0.008718,0.000592,0.000041,0.033822,0.033822,0.033822,0.843967,-7.382638,0.821302,-6.766590,4.509550
6,0.021612,0.010213,-0.021000,-0.010171,0.000612,0.000043,0.037669,0.037669,0.037669,0.881356,-8.339450,0.860326,-7.791940,5.022502
7,0.141794,0.088810,-0.141749,-0.088798,0.000045,0.000012,0.009739,0.009739,0.009739,0.901660,-6.448399,0.901405,-6.445110,4.271682
8,0.133568,0.087659,-0.133543,-0.087653,0.000025,0.000006,0.009300,0.009300,0.009300,0.901660,-6.448399,0.901513,-6.446298,4.078982
9,0.133568,0.087659,-0.133543,-0.087653,0.000025,0.000006,0.009300,0.009300,0.009300,0.901660,-6.448399,0.901513,-6.446298,4.078982


In [126]:
subnet.res_load

,p_mw,q_mvar
0,0.018,0.008718
1,0.018,0.008718
2,0.018,0.008718
3,0.018,0.008718
4,0.018,0.008718
5,0.018,0.008718
6,0.021,0.010171
7,0.129,0.062478
8,0.129,0.062478
9,0.129,0.062478


In [112]:
#dont run pv as generator, model as negaitve load with q=0, for balaeric islands model as load

In [90]:
subnet.res_storage

,p_mw,q_mvar
